In [ ]:
from pyforestscan.calculate import calculate_chm, calculate_fhd, assign_voxels
from pyforestscan.handlers import read_lidar, create_geotiff
import numpy as np
import glob, os

In [28]:
path = "/home/mgallet/Documents/DATA/LIDAR_TEST/processed_V4_fulltiles_avecCSF_002VOX/**/*.npy"
path2 = "/home/mgallet/Documents/DATA/LIDAR_TEST/processed_V2_*/**/*.npy"

In [29]:
data = np.load(glob.glob(path2, recursive=True)[0])

In [ ]:


def create_pdal_pipeline_phase2(input_files, output_file):
    """
    Crée la deuxième phase du pipeline PDAL pour fusionner les tuiles prétraitées
    et appliquer les traitements qui nécessitent une vue globale.

    Parameters:
    ----------
    input_files : list
        Liste des fichiers LAS/LAZ prétraités par la phase 1.
    output_file : str
        Chemin du fichier de sortie final.
    pdal_pipeline_template : list, optional
        Modèle pour les étapes du pipeline PDAL (phase 2 uniquement).
    quality_level : dict, optional
        Paramètres de niveau de qualité à appliquer.

    Returns:
    -------
    dict
        Configuration du pipeline PDAL phase 2.
    """
    pipeline = {"pipeline": []}

    # Ajouter la première tuile avec readers.las
    pipeline["pipeline"].append(
        {
            "type": "readers.las",
            "filename": input_files[0],
            "spatialreference": "EPSG:2154",
            "start": 0,
            "count": 100000,
        }
    )

    # Ajouter les autres tuiles avec readers.las
    for file in input_files[1:]:
        pipeline["pipeline"].append(
            {"type": "readers.las", "filename": file, "spatialreference": "EPSG:2154"}
        )

    # Fusionner les tuiles
    pipeline["pipeline"].append({"type": "filters.merge"})

    # Utiliser hag_nn au lieu de hag_delaunay pour de meilleures performances
    pipeline["pipeline"].append({"type": "filters.hag_nn", "count": 80})

    # Filtrer par hauteur
    pipeline["pipeline"].append(
        {"type": "filters.range", "limits": "HeightAboveGround[0:35]"}
    )

    # Filtrer par classification
    pipeline["pipeline"].append(
        {
            "type": "filters.expression",
            "expression": "((Classification >= 2) && (Classification <= 5))",
        }
    )

    # Ajout de l'étape d'écriture
    pipeline["pipeline"].append(
        {
            "type": "writers.las",
            "filename": output_file,
            "compression": "false",
            "minor_version": "4",
            "forward": "all",
        }
    )

    return pipeline

pip = create_pdal_pipeline_phase2(
    input_files=["/home/mgallet/Documents/DATA/LIDAR_TEST/uncompress/lidar_data/MontBlanc/cheserys/uncompress_LHD_FXX_1001_6551_PTS_C_LAMB93_IGN69.las"],
    output_file="testmerged.las",
)

In [ ]:
import pdal,json
pipeline = pdal.Pipeline(json.dumps(pip))
pipeline.execute()


In [16]:
voxel_resolution = (20, 20, 0.1)
voxels, extent = assign_voxels(data, voxel_resolution)

chm = calculate_chm(data, voxel_resolution, interpolation=None)



ValueError: Point cloud data missing 'X', 'Y', or 'HeightAboveGround' fields.

In [ ]:
# create_geotiff(chm[0], "DATA/LIDAR/test/test/chm.tif", "EPSG:2154", extent)
